In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import gmaps
import gmaps.geojson_geometries
import os
import json
import requests
import pprint as pp



# Import API key
from config import g_key

In [2]:
# File to Load
NYC1_data_to_load = "resources/NYC_Listings1.csv"
NYC2_data_to_load = "resources/NYC_Listings2.csv"
NYC3_data_to_load = "resources/NYC_Listings3.csv"

# Read School and Student Data File and store into Pandas Data Frames
NYC1_data = pd.read_csv(NYC1_data_to_load)
NYC2_data = pd.read_csv(NYC2_data_to_load)
NYC3_data = pd.read_csv(NYC3_data_to_load)

/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Create a full NYC DF
NYC_data = pd.concat([NYC1_data, NYC2_data, NYC3_data], ignore_index=True)
NYC_data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,2.019080e+13,8/7/19,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2.0,1.0,0.0,1.0,0.39
1,3647,https://www.airbnb.com/rooms/3647,2.019080e+13,8/6/19,THE VILLAGE OF HARLEM....NEW YORK !,NaN,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,0.0,1.0,0.0,NaN
2,3831,https://www.airbnb.com/rooms/3831,2.019080e+13,8/6/19,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1.0,1.0,0.0,0.0,4.64
3,5022,https://www.airbnb.com/rooms/5022,2.019080e+13,8/6/19,Entire Apt: Spacious Studio/Loft by central park,NaN,Loft apartment with high ceiling and wood floo...,Loft apartment with high ceiling and wood floo...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.10
4,5099,https://www.airbnb.com/rooms/5099,2.019080e+13,8/6/19,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment is true New York ...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment is true New York ...,none,My neighborhood in Midtown East is called Murr...,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.60


In [4]:
# Review columns
column_names = list(NYC_data.columns)
column_names[:5]

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name']

In [5]:
# Select Columns for clean DataFrame
Clean_NYC = NYC_data[['id','name','neighbourhood',
                      'latitude','longitude','property_type',
                      'room_type','accommodates','amenities',
                      'price','availability_365','number_of_reviews',
                     'number_of_reviews_ltm','review_scores_rating',
                      'review_scores_location','reviews_per_month']]
# Drop null values
Clean_NYC = Clean_NYC.dropna()

# Change price value to float
Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.lstrip('$'))
#Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.lstrip(','))
Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.replace(',',''))
Clean_NYC['price'] = Clean_NYC['price'].astype('float')

# Preview DataFrame
Clean_NYC

,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month
0,2595,Skylit Midtown Castle,Midtown,40.75362,-73.98377,Apartment,Entire home/apt,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",225.0,288.0,46.0,12.0,95.0,10.0,0.39
2,3831,Cozy Entire Floor of Brownstone,Brooklyn,40.68514,-73.95976,Guest suite,Entire home/apt,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",89.0,212.0,274.0,70.0,90.0,10.0,4.64
3,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,40.79851,-73.94399,Apartment,Entire home/apt,1.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Elev...",80.0,0.0,9.0,2.0,93.0,9.0,0.10
4,5099,Large Cozy 1 BR Apartment In Midtown East,Midtown East,40.74767,-73.97500,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Buzzer/w...",200.0,127.0,75.0,10.0,89.0,9.0,0.60
5,5121,BlissArtsSpace!,Brooklyn,40.68688,-73.95596,Apartment,Private room,2.0,"{Wifi,""Air conditioning"",Kitchen,""Pets live on...",60.0,0.0,49.0,0.0,90.0,9.0,0.39
6,5178,Large Furnished Room Near B'way,Manhattan,40.76489,-73.98493,Apartment,Private room,2.0,"{TV,Wifi,""Air conditioning"",""Paid parking off ...",79.0,239.0,434.0,44.0,84.0,10.0,3.48
7,5203,Cozy Clean Guest Room - Family Apt,Upper West Side,40.80178,-73.96723,Apartment,Private room,1.0,"{Internet,Wifi,""Air conditioning"",""Paid parkin...",79.0,0.0,118.0,0.0,98.0,10.0,0.98
8,5222,Best Hideaway,Manhattan,40.72764,-73.97949,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",116.0,322.0,94.0,0.0,95.0,10.0,0.74
9,5238,Cute & Cozy Lower East Side 1 bdrm,Lower East Side,40.71344,-73.99037,Apartment,Entire home/apt,3.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",150.0,0.0,161.0,13.0,94.0,9.0,1.33
10,5295,Beautiful 1br on Upper West Side,Manhattan,40.80316,-73.96545,Apartment,Entire home/apt,2.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Door...",135.0,1.0,53.0,14.0,93.0,9.0,0.42


In [112]:
# GroupBY Neighborhood
neighborhoods_ratings = Clean_NYC.groupby('neighbourhood')['review_scores_location'].mean()
neighborhoods_ratings.sort_values(ascending=False, inplace=True)
neighborhoods_ratings.head()
# USE GOOGLE PLACES API TO CREATE A MAP THATS SPLIT

neighbourhood
Times Square/Theatre District    10.0
Grant City                       10.0
City Island                      10.0
Oakwood                          10.0
Woodlawn                         10.0
Name: review_scores_location, dtype: float64

In [37]:
neighborhoods_list = list(neighborhoods_ratings.index)
neighborhoods_list[:10]
len(neighborhoods_list)

190

In [9]:
# Create a Dictionary with neighbourhood DataFrames
d = {}
for city in neighborhoods_list:
    checker = (Clean_NYC['neighbourhood'] == city)
    city_df = Clean_NYC.where(checker)
    city_df = city_df.dropna()
    d[city] = city_df


In [161]:
final_neighborhood_list = list(d.keys())
final_neighborhood_list.sort()
final_neighborhood_list

['Allerton-Pelham Gardens',
 "Annadale-Huguenot-Prince's Bay-Eltingville",
 'Astoria',
 'Bath Beach',
 'Battery Park City-Lower Manhattan',
 'Bay Ridge',
 'Bayside-Bayside Hills',
 'Bedford',
 'Bedford Park-Fordham North',
 'Belmont',
 'Bensonhurst West',
 'Borough Park',
 'Brighton Beach',
 'Bronxdale',
 'Brooklyn',
 'Brooklyn Heights-Cobble Hill',
 'Brooklyn Navy Yard',
 'Brownsville',
 'Bushwick',
 'Canarsie',
 'Carroll Gardens',
 'Castle Hill ',
 'Castleton Corners',
 'Chelsea',
 'Chinatown',
 'City Island',
 'Civic Center',
 'Claremont',
 'Clinton Hill',
 'College Point',
 'Columbia Street Waterfront',
 'Concord',
 'Concourse',
 'Concourse Village',
 'Coney Island',
 'Corona',
 'Country Club',
 'Crotona',
 'Crown Heights',
 'DUMBO',
 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill',
 'Ditmars / Steinway',
 'Dongan Hills',
 'Downtown Brooklyn',
 'Dyker Heights',
 'East Elmhurst',
 'East Flatbush',
 'East Harlem',
 'East New York',
 'East Village',
 'Eastchester',
 'Eastchester-Ed

### CHANGING NAMES AND UPDATING DICTIONARY

In [115]:
# Function to Change Names
def change_name(self,key,new_key):
    value = self[key]
    self.pop(key)
    self[new_key] = value

In [118]:
# Functino to Update Dictionary
def title_name_change(self,old_key, new_key):
    change_name(self,old_key,new_key)
    dataframe = self[new_key]
    dataframe['new neighborhood'] = new_key

In [263]:
title_name_change(d,'Civic Center',"SoHo-TriBeCa-Civic Center-Little Italy")
d['SoHo-TriBeCa-Civic Center-Little Italy'].head()

,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month,new neighborhood
7476,5727850.0,Beautiful Room Near S. St. Seaport,Civic Center,40.70854,-73.99992,Apartment,Private room,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",70.0,297.0,12.0,2.0,92.0,10.0,0.27,SoHo-TriBeCa-Civic Center-Little Italy
14805,12004744.0,Spacious Lower Manhattan Apt!,Civic Center,40.71127,-73.99838,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",159.0,0.0,3.0,0.0,100.0,10.0,0.11,SoHo-TriBeCa-Civic Center-Little Italy
15569,12810452.0,Charming 1BR in heart of Chinatown,Civic Center,40.71114,-73.99874,Apartment,Private room,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",99.0,0.0,3.0,0.0,87.0,9.0,0.08,SoHo-TriBeCa-Civic Center-Little Italy
26524,21384151.0,"Quiet, Private BR (Queen Bed): Views of BK Bridge",Civic Center,40.70940,-74.00145,Apartment,Private room,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Pets allo...",96.0,0.0,4.0,0.0,100.0,10.0,0.18,SoHo-TriBeCa-Civic Center-Little Italy
30152,23784779.0,Downtown private room (Queen+Twin) + BREAKFAST,Civic Center,40.71179,-73.99848,Apartment,Private room,3.0,"{Wifi,""Air conditioning"",Kitchen,""Paid parking...",160.0,50.0,136.0,96.0,98.0,10.0,8.21,SoHo-TriBeCa-Civic Center-Little Italy


In [266]:
# ADDING LISTS TOGETHER

d['SoHo-TriBeCa-Civic Center-Little Italy'] = d['SoHo-TriBeCa-Civic Center-Little Italy'].append(d['Little Italy'], ignore_index=False)


In [267]:
# adding new neighborhood name
d['SoHo-TriBeCa-Civic Center-Little Italy']['new neighborhood'] = 'SoHo-TriBeCa-Civic Center-Little Italy'




In [268]:
d['SoHo-TriBeCa-Civic Center-Little Italy']

,accommodates,amenities,availability_365,id,latitude,longitude,name,neighbourhood,new neighborhood,number_of_reviews,number_of_reviews_ltm,price,property_type,review_scores_location,review_scores_rating,reviews_per_month,room_type
7476,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",297.0,5727850.0,40.70854,-73.99992,Beautiful Room Near S. St. Seaport,Civic Center,SoHo-TriBeCa-Civic Center-Little Italy,12.0,2.0,70.0,Apartment,10.0,92.0,0.27,Private room
14805,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",0.0,12004744.0,40.71127,-73.99838,Spacious Lower Manhattan Apt!,Civic Center,SoHo-TriBeCa-Civic Center-Little Italy,3.0,0.0,159.0,Apartment,10.0,100.0,0.11,Entire home/apt
15569,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",0.0,12810452.0,40.71114,-73.99874,Charming 1BR in heart of Chinatown,Civic Center,SoHo-TriBeCa-Civic Center-Little Italy,3.0,0.0,99.0,Apartment,9.0,87.0,0.08,Private room
26524,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Pets allo...",0.0,21384151.0,40.70940,-74.00145,"Quiet, Private BR (Queen Bed): Views of BK Bridge",Civic Center,SoHo-TriBeCa-Civic Center-Little Italy,4.0,0.0,96.0,Apartment,10.0,100.0,0.18,Private room
30152,3.0,"{Wifi,""Air conditioning"",Kitchen,""Paid parking...",50.0,23784779.0,40.71179,-73.99848,Downtown private room (Queen+Twin) + BREAKFAST,Civic Center,SoHo-TriBeCa-Civic Center-Little Italy,136.0,96.0,160.0,Apartment,10.0,98.0,8.21,Private room
33440,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",269.0,27016768.0,40.71389,-74.00484,Luxury Studio 5 min to World Trade & BK Bridge,Civic Center,SoHo-TriBeCa-Civic Center-Little Italy,37.0,35.0,199.0,Serviced apartment,10.0,99.0,2.89,Entire home/apt
460,2.0,"{Wifi,""Air conditioning"",Kitchen,""Paid parking...",153.0,164989.0,40.72351,-73.99683,"*SoHo: Clean, Safe, Private, Peaceful Bedroom ...",Soho,SoHo-TriBeCa-Civic Center-Little Italy,58.0,21.0,140.0,Apartment,10.0,93.0,0.59,Private room
986,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",247.0,405408.0,40.72057,-73.99976,Magazine SOHO Studio Loft. Read our reviews!,Soho,SoHo-TriBeCa-Civic Center-Little Italy,135.0,23.0,225.0,Loft,10.0,96.0,1.53,Entire home/apt
1321,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",311.0,587740.0,40.72372,-74.00277,Designer open space in TriBeCa Soho 1000sq ft,Soho,SoHo-TriBeCa-Civic Center-Little Italy,11.0,3.0,199.0,Loft,10.0,95.0,0.13,Entire home/apt
2172,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",22.0,1039813.0,40.72575,-74.00093,Soho Studio w/ exposed brick & lively plants,Soho,SoHo-TriBeCa-Civic Center-Little Italy,24.0,8.0,147.0,Apartment,10.0,95.0,0.46,Entire home/apt


In [234]:
final_neighborhood_list

['Allerton-Pelham Gardens',
 "Annadale-Huguenot-Prince's Bay-Eltingville",
 'Astoria',
 'Bath Beach',
 'Battery Park City-Lower Manhattan',
 'Bay Ridge',
 'Bayside-Bayside Hills',
 'Bedford',
 'Bedford Park-Fordham North',
 'Belmont',
 'Bensonhurst West',
 'Borough Park',
 'Brighton Beach',
 'Bronxdale',
 'Brooklyn',
 'Brooklyn Heights-Cobble Hill',
 'Brooklyn Navy Yard',
 'Brownsville',
 'Bushwick',
 'Canarsie',
 'Carroll Gardens',
 'Castle Hill ',
 'Castleton Corners',
 'Chelsea',
 'Chinatown',
 'City Island',
 'Civic Center',
 'Claremont',
 'Clinton Hill',
 'College Point',
 'Columbia Street Waterfront',
 'Concord',
 'Concourse',
 'Concourse Village',
 'Coney Island',
 'Corona',
 'Country Club',
 'Crotona',
 'Crown Heights',
 'DUMBO',
 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill',
 'Ditmars / Steinway',
 'Dongan Hills',
 'Downtown Brooklyn',
 'Dyker Heights',
 'East Elmhurst',
 'East Flatbush',
 'East Harlem',
 'East New York',
 'East Village',
 'Eastchester',
 'Eastchester-Ed

In [150]:
new_neighborhood_json_list = []

with open('resources/nyc-neighborhoods.geo.json') as f:
    geometry = json.load(f)
    
#     for key,value in geometry.items():
#         print(geometry['features'][0])

    for row in geometry['features']:
        city = row['properties']['name']
        if city not in new_neighborhood_json_list:
            new_neighborhood_json_list.append(city)

len(new_neighborhood_json_list)

188

In [262]:
for row in new_neighborhood_json_list:
    if 'Civic Center' in row:
        print(row)

SoHo-TriBeCa-Civic Center-Little Italy


In [167]:
# DATA MUNGING TO FIX CITY MAP
final_neighborhood_list
new_neighborhood_json_list
new_missing_cities = []
new_extra_json_cities = []


for city in final_neighborhood_list:
    if city not in new_neighborhood_json_list:
        new_missing_cities.append(city)
        
for city in new_neighborhood_json_list:
    if city not in final_neighborhood_list:
        new_extra_json_cities.append(city)

print('MISSING CITIES TO ADD', len(new_missing_cities))
print('EXTRA CITIES IN GRAPH', len(new_extra_json_cities))
new_missing_cities



#ALPHABETIZE Lists
new_neighborhood_json_list.sort()
new_missing_cities.sort()
len(new_neighborhood_json_list)


checker_df = pd.DataFrame({'missing cities': pd.Series(new_missing_cities), 
              'json list': pd.Series(new_neighborhood_json_list),
             'extra json to use': pd.Series(new_extra_json_cities)})
checker_df

#'Williamsburg' in new_neighborhood_json_list

MISSING CITIES TO ADD 118
EXTRA CITIES IN GRAPH 118


,missing cities,json list,extra json to use
0,Brooklyn,Allerton-Pelham Gardens,Arden Heights
1,Brooklyn Navy Yard,Annadale-Huguenot-Prince's Bay-Eltingville,Auburndale
2,Bushwick,Arden Heights,Baisley Park
3,Carroll Gardens,Astoria,Bellerose
4,Castle Hill,Auburndale,Bensonhurst East
5,Castleton Corners,Baisley Park,Breezy Point-Belle Harbor-Rockaway Park-Broad ...
6,Chelsea,Bath Beach,Briarwood-Jamaica Hills
7,City Island,Battery Park City-Lower Manhattan,Bushwick North
8,Civic Center,Bay Ridge,Bushwick South
9,Claremont,Bayside-Bayside Hills,Cambria Heights


# THE NEW NEIGHBORHOOD COLUMN AN

In [ ]:
# # Configure gmaps with API key
# gmaps.configure(api_key=g_key)
# # Create a Gmaps figure
# fig = gmaps.figure()

# locations = Clean_NYC[['latitude','longitude']]
# weights = Clean_NYC['review_scores_location']
# # Create the Heat Layer Object 
# heat_layer = gmaps.heatmap_layer(locations=locations, weights=weights, dissipating=True)
# heat_layer.point_radius = 4
# heat_layer.gradient = [
#     'gray',
#     'black',
#     'blue'
# ]
# # Add Heat Layer to Figure
# fig.add_layer(heat_layer)

# fig

In [ ]:
# new_york_coordinates = (40.75, -74.00)
# #gmaps.figure(center=new_york_coordinates, zoom_level=12)

# countries_geojson = gmaps.geojson_geometries.load_geometry('us-counties')

# fig = gmaps.figure(center=new_york_coordinates, zoom_level=12)
# gini_layer = gmaps.geojson_layer(countries_geojson)
# fig.add_layer(gini_layer)
# #fig
